In [30]:
import os

from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['COHERE_API_KEY'] = os.getenv('COHERE_API_KEY')

In [31]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

In [32]:
# Carregar modelos Open AI - Embeddings e Chat
embeddings_model = OpenAIEmbeddings(model='text-embedding-ada-002')
llm = ChatOpenAI(model='gpt-3.5-turbo', max_tokens=300)

In [33]:
# Carregar o PDF
pdf_link = '../../pdfs/weight-loss.pdf'

loader = PyPDFLoader(pdf_link, extract_images=False)

pages = loader.load_and_split()

len(pages)

8

In [34]:
# Separar em chunks
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=4000,
  chunk_overlap=20,
  length_function=len,
  add_start_index=True,
)

chunks = text_splitter.split_documents(pages)

In [35]:
# Salvar os chunks no Vector DB
vectordb = Chroma.from_documents(
  chunks,
  embedding=embeddings_model,
  persist_directory='naiveDB',
)

In [36]:
# Carregar o DB
naive_retriever = vectordb.as_retriever(search_kwargs={'k': 10})

In [37]:
# Criar o reranker
reranker = CohereRerank(
  top_n=3,
  model='rerank-multilingual-v3.0',
)

compressor_retriever = ContextualCompressionRetriever(
  base_compressor=reranker,
  base_retriever=naive_retriever,
)

In [38]:
TEMPLATE = """
  Você é um especialista em nutrição e saúde. Sua tarefa é responder perguntas sobre o documento fornecido.
  Se a pergunta do usuário não está relacionada ao documento, responda que não sabe.

  Documento:
  {context}

  Pergunta:
  {question}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [39]:
setup_retrieval = RunnableParallel(
  {
    'question': RunnablePassthrough(),
    'context': compressor_retriever
  },
)

output_parser = StrOutputParser()

compressor_retrieval_chain = setup_retrieval | rag_prompt | llm | output_parser

In [40]:
compressor_retrieval_chain.invoke('Como perder peso de maneira rápida e saudável?')

'A perda de peso saudável deve ocorrer quando você consome menos calorias do que queima. Aumentar a atividade física enquanto limita a ingestão de calorias aumentará sua taxa de perda de peso. Além disso, aumentar a atividade física também ajudará a manter seu peso após a perda de peso. É importante discutir os níveis adequados de calorias e tamanhos de porções com seu nutricionista. Além disso, seguir as diretrizes fornecidas no documento, como manter um diário alimentar e de atividades físicas, pesar-se uma vez por semana, comer devagar, escolher alimentos com menos gordura e açúcar, e beber bastante líquido são dicas importantes para perder peso de maneira saudável e eficaz.'